# Analysis of Amazon Product Data

After construction of the graph (product-id and link to cross-buys), the following topics will be investigated:
* correlation between in-degrees and salesrank
* and some further smart questions... influence of number of reviews?, are there different sub-graphs like produkt categories?
* 
* 
* 
Data source: https://snap.stanford.edu/data/amazon-meta.html

Data import

In [3]:
# Import graphframes (from Spark-Packages)

from graphframes import *
from pyspark.sql.functions import monotonically_increasing_id, desc


# Get the product data (already pre-processed data in table format)
products = spark.read.csv("/FileStore/tables/ProductData_small.csv", sep=";", header=True)

# The the cross-buy data (similar products)
links = spark.read.csv("/FileStore/tables/LinkData_small.csv", sep=";", header=True)

Out[ 67 ]: 52158

How many products are we dealing with here?

In [5]:
products.count()

Out[ 61 ]: 10611

quick check of the data formats

In [7]:
display(products.take(3))

Id,ASIN,Group,Salesrank,Reviews,Title
21,0790747324,DVD,795,140,The Time Machine
84,B000063W82,Video,780,14,The Best of Schoolhouse Rock! - 30th Anniversary Edition
252,B0000262WI,Music,581,82,The Köln Concert


In [8]:
display(links.take(3))

src,dest
0790747324,B00007JMD8
0790747324,6305350221
0790747324,B00004RF9B


The graph will be constructed based on the product data. Each product is identified through its unique *ASIN* (Amazon Standard Identification Number) code

GraphFrame library required *ASIN* to be renamed to *id*, but be careful, there is already an "Id" in the DataFrame

In [10]:
# rename "Id" to "Id-nu" to avoid any confusion
# rename "ASIN" to "id"
ProductVertices = products.withColumnRenamed("Id", "Id-nu").distinct()
ProductVertices = ProductVertices.withColumnRenamed("ASIN", "id").distinct()

# check if done correctly
display(ProductVertices.take(3))

Id-nu,id,Group,Salesrank,Reviews,Title
13260,B000002TIR,Music,2004,16,French Kiss
13683,0140281916,Book,2877,16,Bargaining for Advantage : Negotiation Strategies for Reasonable People
30773,B000001EYO,Music,3579,31,The Big Lebowski: Original Motion Picture Soundtrack


How does the Edge table look like?

In [12]:
display(links.take(1))

src,dest
0790747324,B00007JMD8


Although not always strictly required, it's good practice to have an ID column for the tables. 

Since the product links does not have one yet, create and additional ID:

In [14]:
# 2. Create Edges
links = links.withColumn("link-id", monotonically_increasing_id())

display(links.take(3))

src,dest,link-id
0790747324,B00007JMD8,0
0790747324,6305350221,1
0790747324,B00004RF9B,2


Salesrank and number of reviews are read as string. Change type to int for calculations.

In [16]:
# cast the arrival delay column 'ARR_DELAY' from String to int and rename the column to 'delay'
# TODO: put in the new column name below
ProductVertices = ProductVertices.withColumn('Salesrank', ProductVertices.Salesrank.cast('int'))
ProductVertices = ProductVertices.withColumn('Reviews', ProductVertices.Reviews.cast('int'))
display(ProductVertices.take(1))

Id-nu,id,Group,Salesrank,Reviews,Title
13260,B000002TIR,Music,2004,16,French Kiss


Rename links to similar products ar *src* and *dst*

In [18]:
# rename destination "dest" to "dst"
links = links.withColumnRenamed('dest', 'dst')

# let's keep only the most important columns from the whole dataframe
#tripEdges = departureDelays.select("tripid", "src", "dst", "origin_city_name", "dest_city_name", "delay")


Now comes the moment of truth! Let's see if the graph can be constructed!

In [20]:
# 2. Build the GraphFrame based on products and links

ProductGraph = GraphFrame(ProductVertices, links)

# print a sample of the vertices and edges
print("Products: %d" % ProductGraph.vertices.count())
print
print("SAMPLE PRODUCT DATA:")
print(ProductGraph.vertices.take(1))
print

print("Links: %d" % ProductGraph.edges.count())
print
print("SAMPLE LINK DATA:")
print(ProductGraph.edges.take(1))
print

Products: 10611
SAMPLE PRODUCT DATA:
[Row(Id-nu='13260', id='B000002TIR', Group='Music', Salesrank=2004, Reviews=16, Title='French Kiss')]
Links: 52158
SAMPLE LINK DATA:
[Row(src='0790747324', dst='B00007JMD8', link-id=0)]
 Out[ 11 ]: <function print>

The Graph is constructed. Do the fancy analyses!

1) assess the number of in-degress per product and compare it to the salesrank

Plot chunk 24 in the following way:

you can look at the distribution of the inDegree-values by clicking on the icon to the right of 'Raw table' and then chosing a histogram, values = 'inDegree' and  30 bins

We can see that most of the inDegree values are between 0 and 2

In [24]:
display(ProductGraph.inDegrees.sort('inDegree', ascending=False))
#inDegRank = ProductGraph.inDegrees.sort('inDegree', ascending=False)

id,inDegree
B00008LDNZ,78
B00005JMAH,55
B00023P4I8,53
B00003CXCT,52
B000634DCW,45
B000068DBC,37
6304765266,36
B00005ATZT,34
B00001QEE7,33
B00005JMIJ,33


In [25]:
maxNumInDeg = ProductGraph.inDegrees.groupBy().max('inDegree')
display(maxNumInDeg)

maxNumInDegValue = str(maxNumInDeg.collect()[0]['max(inDegree)'])
#print(maxNumInDegValue)

maxInDegreeProduct = ProductGraph.inDegrees.filter('inDegree = {}'.format(maxNumInDegValue))
display(maxInDegreeProduct)


id,inDegree
B00008LDNZ,78


In [26]:
# join back to get product info based on max inDegree
ProductMaxInDegree = ProductGraph.vertices.join(maxInDegreeProduct, 'id')
display(ProductMaxInDegree)




id,Id-nu,Group,Salesrank,Reviews,Title,inDegree
B00008LDNZ,548091,DVD,110,83,Laura,78


Plot the following chunk in this way:
Plot this as a bar chart with 'Plot Options':
 - Keys: InDegree
 - Values: Salesrank
 
Unfortunately you can't chose to have a logarithmic y-axis, but you do see that at least for low in-degree values the sales rank is rather bad.

In [28]:
# repeat exercise from above but with entire list
selection = ProductGraph.inDegrees.sort('inDegree', ascending=False)
#display(selection)

ProductSortInDeg = ProductGraph.vertices.join(selection, 'id').sort('inDegree', ascending=False)
display(ProductSortInDeg)

id,Id-nu,Group,Salesrank,Reviews,Title,inDegree
B00008LDNZ,548091,DVD,110,83,Laura,78
B000068DBC,546689,DVD,109,633,Pulp Fiction (Collector's Edition),37
6304765266,97579,DVD,351,152,While You Were Sleeping,36
B00005ATZT,547040,DVD,1020,140,The Fugitive (Special Edition),34
B00001QEE7,104775,DVD,49,160,The Little Mermaid (Limited Issue),33
6305368171,276371,DVD,236,500,You've Got Mail,33
B00003CXA2,547272,DVD,185,530,Forrest Gump,30
630522577X,548182,DVD,253,180,My Fair Lady,29
B000059HB7,4388,DVD,1877,79,Rio Bravo,29
B000022TSH,544390,DVD,1108,154,Chinatown,27


Plot the following chunk in this way:
Plot this as a bar chart with 'Plot Options':
 - Keys: OutDegree
 - Values: Salesrank

In [30]:
# repeat exercise from above for outDegree
selection = ProductGraph.outDegrees.sort('outDegree', ascending=True)
#display(selection)

ProductSortOutDeg = ProductGraph.vertices.join(selection, 'id').sort('OutDegree', ascending=True)
display(ProductSortOutDeg)

outDegree is rather boring, since most of the products get assigned 5 products by default that are presumingly similar, as you can see here:

In [32]:
display(ProductSortOutDeg.groupBy("outDegree").count())

outDegree,count
5,10343
4,43
3,48
2,43
1,41


plot the output of chunk 31 in the following way:
chose a bar chart with options:
  - keys: Reviews
  - values: Salesrank
  
and apply to all rows.
We can see that the less reviews a product has, the less likely it is to be a top selling product (the causal orientation might also be inverse here :) )

In [34]:
# relation of number of reviews and sales rank:
display(ProductGraph.vertices.select("Reviews", "Salesrank").sort("Reviews", ascending = False))


Reviews,Salesrank
5545,110
5539,2135
5539,420
5039,1533
5039,170
5034,746
4924,404
4924,661
4924,327
3839,3221


trying to find products which have an edge to another product which in turn has an edge to the first product:

In [36]:
ProductsWithBalancedSimilarity = ProductGraph.find("(v1)-[e1]->(v2); (v2)-[e2]->(v1)")
display(ProductsWithBalancedSimilarity)

v1,e1,v2,e2
"List(21, 0790747324, DVD, 795, 140, The Time Machine)","List(0790747324, B00007JMD8, 0)","List(114681, B00007JMD8, DVD, 1076, 104, Journey to the Center of the Earth)","List(B00007JMD8, 0790747324, 11138)"
"List(84, B000063W82, Video, 780, 14, The Best of Schoolhouse Rock! - 30th Anniversary Edition)","List(B000063W82, 156949407X, 5)","List(239294, 156949407X, Video, 1223, 24, Schoolhouse Rock! - Grammar Rock)","List(156949407X, B000063W82, 22808)"
"List(296, 0385504209, Book, 19, 3049, The Da Vinci Code)","List(0385504209, 0671027360, 20)","List(376858, 0671027360, Book, 31, 1552, Angels & Demons)","List(0671027360, 0385504209, 34818)"
"List(296, 0385504209, Book, 19, 3049, The Da Vinci Code)","List(0385504209, 0671027387, 22)","List(424705, 0671027387, Book, 84, 429, Deception Point)","List(0671027387, 0385504209, 38171)"
"List(448, 0312966091, Book, 607, 141, Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel))","List(0312966091, 0312990456, 40)","List(471848, 0312990456, Book, 304, 414, One for the Money (A Stephanie Plum Novel))","List(0312990456, 0312966091, 41794)"
"List(448, 0312966091, Book, 607, 141, Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel))","List(0312966091, 0312971346, 41)","List(104366, 0312971346, Book, 1401, 264, High Five (A Stephanie Plum Novel))","List(0312971346, 0312966091, 10118)"
"List(448, 0312966091, Book, 607, 141, Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel))","List(0312966091, 0312976275, 42)","List(475036, 0312976275, Book, 787, 324, Hot Six : A Stephanie Plum Novel (A Stephanie Plum Novel))","List(0312976275, 0312966091, 41970)"
"List(448, 0312966091, Book, 607, 141, Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel))","List(0312966091, 0312980140, 43)","List(158327, 0312980140, Book, 1084, 274, Seven Up (A Stephanie Plum Novel))","List(0312980140, 0312966091, 15295)"
"List(457, B0000296JB, Music, 2439, 545, Make Yourself)","List(B0000296JB, B00005QG9J, 45)","List(119794, B00005QG9J, Music, 2488, 622, Morning View)","List(B00005QG9J, B0000296JB, 11691)"
"List(480, B0000296J9, Music, 1310, 42, Gunfighter Ballads & Trail Songs)","List(B0000296J9, B0000026AG, 50)","List(219235, B0000026AG, Music, 2015, 26, Johnny Horton - Greatest Hits)","List(B0000026AG, B0000296J9, 21158)"


how many product-pairs reference each other? can we just calculate it like this?

In [38]:
ProductsWithBalancedSimilarity.count()

Out[ 60 ]: 11202

does this mean, we have 11202 product-pairs that reference each other? Let's test if our data frame of mutually referencing products lists the pairs once or twice by looking at an example:

In [40]:
display(ProductsWithBalancedSimilarity.filter("(v1.id = '0790747324' and v2.id = 'B00007JMD8') or (v2.id = '0790747324' and v1.id = 'B00007JMD8')"))

v1,e1,v2,e2
"List(21, 0790747324, DVD, 795, 140, The Time Machine)","List(0790747324, B00007JMD8, 0)","List(114681, B00007JMD8, DVD, 1076, 104, Journey to the Center of the Earth)","List(B00007JMD8, 0790747324, 11138)"
"List(114681, B00007JMD8, DVD, 1076, 104, Journey to the Center of the Earth)","List(B00007JMD8, 0790747324, 11138)","List(21, 0790747324, DVD, 795, 140, The Time Machine)","List(0790747324, B00007JMD8, 0)"


So the pairs are listed twice, which kind of makes sense, seeing how we constructed the data frame. This means there are 11202/2 = 5601 product-pairs that reference each other.

In [42]:
connectedComp = ProductGraph.stronglyConnectedComponents(maxIter=3)
#connectedComp.select("id", "component").groupBy("component").count().sort(desc("count")).show()

#ProductGraph.connectedComponents()
display(connectedComp)

In [43]:
connectedComp.groupBy("component", "Group").count().sort(desc("count")).show()